# SPX volatility

The aim of this notebook is to investigate the volatility of SPX and to see whether the measured volatility matches the implied volatilities observed through options. The following steps will be undertaken:


- **Day count**: I investigate whether non-trading days have lower volatility than trading days and whether changes on the last trading day before a non trading day have a different volatility than other trading days (also based on a comment in the book)
- **Long history**: I compare different moving average windows in order to see what can be said about the recommended window of 90 or 180 days
- **Volatility growth**: In many (risk neutral!) calculations, uncertainty grows as `sqrt(t)`. This is compared with the (real world) increase in volatilities. Since the volatiliy does not change from the real world to risk neutral, this relationship should also hold on observed data.
- **Implied volatilities**: A simple comparison between implied and (backward-)measured volatilities.
- **Volatility smiles**: A comparison of implied volatilities for different strikes to compare against the expectation from the book.

A version of the notebook is available as html file since github sometimes cannot properly display notebooks.

In [1]:
from datetime import datetime
from io import StringIO
import os

import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.offline.offline import iplot
from scipy.stats import spearmanr, norm
from sklearn.linear_model import LinearRegression, HuberRegressor, RANSACRegressor
from sklearn.preprocessing import PolynomialFeatures
import requests


## Day count conventions

In [2]:
csv = requests.get("https://raw.githubusercontent.com/o1i/hull/main/data/2012-12-13_spx_historic.csv").content.decode("utf-8")


In [3]:
spx_hist = pd.read_csv(StringIO(csv))
dt_fmt = "%Y-%m-%d"
spx_hist["date_dt"] = spx_hist["date"].map(lambda x: datetime.strptime(x, dt_fmt))
spx_hist.sort_values("date_dt", inplace=True)
spx_hist.set_index("date_dt", inplace=True)
spx_hist["weekday"] = spx_hist.index.map(lambda x: x.strftime("%a"))
spx_hist["log_return"] = np.log10(spx_hist["close"] / spx_hist["close"].shift(1))

# The first 15 years or so open = close --> to be excluded
first_close_unlike_open = list(~(spx_hist["open"] == spx_hist["close"])).index(True)
spx_hist_short = spx_hist[first_close_unlike_open:]
intra_day = np.log10(spx_hist_short["close"] / spx_hist_short["open"])

### Intra Day moves

In [4]:
fig = go.Figure(layout_yaxis_range=[-0.01,0.01], 
                layout_title="One-day log10-returns by weekday")
for wd in ["Mon", "Tue", "Wed", "Thu", "Fri"]:
    fig.add_trace(go.Box(y=intra_day[spx_hist_short["weekday"] == wd], name=wd))
iplot(fig)

Interestingly, the median values are increasing over the week (aka relatively more positive movements towards the end of the week)

In [ ]:
fig = go.Figure(layout_yaxis_range=[0,0.01], 
                layout_title="Absolute one day log10 returns by weekday")
for wd in ["Mon", "Tue", "Wed", "Thu", "Fri"]:
    fig.add_trace(go.Box(y=intra_day[spx_hist_short["weekday"] == wd].map(lambda x: abs(x)), name=wd))
iplot(fig)

While both Q3 as well as the upper fence is lower on Friday, it does not seem to fundamentally change the picture compared to other days. Also assuming that this pattern is so trivial it would be exploited until it no longer was a pattern, I will not treat Fridays differently in what follows.

### Trading vs non-trading days

Since in the data available to me the "implied" prices at the end of non-business days were not availabe, I will compare the following:

- The close of day `d` is compared to the open of `d+3` for Mondays, Tuesdays and Fridays.
- Only two-day breaks over the weekend will be considered for simplicity. Any three-day weekend or a non-trading day in the middle of the week will be ignored.
- Only the pattern over the entire period is analysed. Changes in this behaviour could be academically of interest, but not done here since not at the heart of what this notebook should be (implied volatilities).

In [ ]:
breaks = spx_hist_short.copy()
breaks[["wd_1", "open_1"]] = breaks[["weekday", "open"]].shift(-1)
breaks[["wd_3", "open_3"]] = breaks[["weekday", "open"]].shift(-3)
breaks = breaks[((breaks["weekday"] == "Mon") & (breaks["wd_3"] == "Thu")) | 
                ((breaks["weekday"] == "Tue") & (breaks["wd_3"] == "Fri")) |
                ((breaks["weekday"] == "Fri") & (breaks["wd_1"] == "Mon"))]
breaks["open_after"] = np.where(breaks["weekday"] == "Fri", breaks["open_1"], breaks["open_3"])
gap = np.log10(breaks["open_after"] / breaks["close"])

In [ ]:
fig = go.Figure(layout_yaxis_range=[-0.03, 0.03], 
                layout_title="log10(open_(d+2) / close(d)) starting on different weekdays")
for wd in ["Mon", "Tue", "Fri"]:
    fig.add_trace(go.Box(y=gap[breaks["weekday"] == wd], name=wd))
iplot(fig)

As expected, there is significantly more movement over trading periods than in non-trading periods. I will therefore, as suggested by Hull, ignore non-trading days but treat fridays as any other day. Thus the holes in the time series do not require special treatment. Just as a confirmation, I will look at close-to-close variability that now should be slightly larger for mondays that incorporate the small Friday close to Monday open volatility.

In [ ]:
fig = go.Figure(layout_yaxis_range=[0,0.025], 
                layout_title="Close-to-close absolute 1-day backward looking log10-returns for consecutive trading days")
for wd in ["Mon", "Tue", "Wed", "Thu", "Fri"]:
    fig.add_trace(go.Box(y=spx_hist_short.loc[spx_hist_short["weekday"] == wd, "log_return"].map(lambda x: abs(x)), name=wd))
iplot(fig)

As expected the values are a tad higher, but by surprisingly little.

What is not done here is to see whether on bank holidays (which may be idiosyncratic to U.S. stocks) there is more volatility than on weekends (that are the same in most major market places). One hypothesis could be that the reduced volatility is due to less information on those days, which would be more the case for weekends than for country-specific days off.

Since we can now look at close to close movements, the whole time series becomes usable.

## Past volatility to predict future volatility

In [ ]:
# Assumption: 252 business days per year, i.e. 21 per month
def std_trace(n_month: int, col: str, name: str, backward: bool = True):
    n = 21*n_month
    window = n if backward else pd.api.indexers.FixedForwardWindowIndexer(window_size=n)
    return go.Scatter(
        x=spx_hist.iloc[::5].index,
        y=spx_hist["log_return"].rolling(window).std().values[::5],
        mode="lines",
        marker={"color":col},
        name=name,
        text=[f"Index: {i}" for i in range(len(spx_hist.index))]
    )
#trace_bw_1m = std_trace(1, "#762a83", "BW 1m", True)
trace_bw_3m = std_trace(3, "#9970ab", "BW 3m", True)
trace_bw_6m = std_trace(6, "#c2a5cf", "BW 6m", True)
#trace_bw_12m = std_trace(12, "#e7d4e8", "BW 12m", True)
#trace_fw_1m = std_trace(1, "#1b7837", "FW 1m", False)
trace_fw_3m = std_trace(3, "#5aae61", "FW 3m", False)
trace_fw_6m = std_trace(6, "#a6dba0", "FW 6m", False)
#trace_fw_12m = std_trace(12, "#d9f0d3", "FW 12m", False)

layout = {
    'showlegend': True,
    "title": "Little agreement of backward and forward standard deviation",
    "xaxis": {"title": "Date"},
    "yaxis": {"title": "Std of daily close-to-close log-returns"}
}

fig = {
    'layout': layout,
    'data': [#trace_bw_1m, 
             trace_bw_3m, 
             trace_bw_6m, 
             #trace_bw_12m,
             #trace_fw_1m, 
             trace_fw_3m, 
             trace_fw_6m, 
             #trace_fw_12m
    ],
}

iplot(fig)


It appears as if except in the stationary case (ca 2012-2015) the past volatility does a surprisingly bad job of predicting future volatility (with obvious implications for options pricing). While one could do formal statistical tests, I believe a scatterplot and maybe an R2 or so will get me closer to a feeling about what is actually happening.

All four trailing windows can be used as estimators for all the leading windows, leading to 16 possible combinations. Also, these windows are available on every trading day and therefore looking at windows on every day would lead to strong dependencies whereas arbitrarily choosing how to split the data into disjunct parts may also lead to variance inflation.

I will therefore for one example (6m back, 6m forward) compare the variance of the R2 estimator introduced by the choice of windows, and if sufficiently small pick the canonical non-overlapping windowms for every combination of leading and trailing window size for further analysis. The expectation is that the plot of offset vs R2 is nearly constant and has (almost) the same values for offset 0 as for offset 252-1.


In [ ]:
n = int(252/2)
backward = spx_hist["log_return"].rolling(n).std()
forward = spx_hist["log_return"].rolling(pd.api.indexers.FixedForwardWindowIndexer(window_size=n)).std()
valid = backward.notna() & forward.notna()
backward = backward[valid]
forward = forward[valid]
index = np.array(range(len(forward)))

def get_r2(offset: int, window: int):
    x = backward[offset::window].values.reshape([-1, 1])
    y = forward[offset::window]
    model = LinearRegression()
    model.fit(x, y)
    return model.score(x, y)

In [ ]:
window = 252
fig = go.Figure(layout_yaxis_range=[0,0.5], 
                layout_title="Expanatory power measured in R2 depends heavily on window offset",
                layout_xaxis_title="Offset (in trading days)",
                layout_yaxis_title="R2 of forward std regressed on backward std")
fig.add_trace(go.Scatter(x=list(range(window)), y=[get_r2(i, window) for i in range(window)], mode="markers+lines"))
iplot(fig)

Clearly only the second assumption holds. It appears as if R2 is extremely sensitive to the offset. For example, comparing offset 0 vs 50 R2 drops from about 50% to 10% explained variance, which would mean that deciding on a backwards window size to predict a certain future window of volatility would have to somehow take into account all possible offsets. To confirm, let's have a closer look at this specific example.

In [ ]:
window = 252
offset_0 = 0
offset_1 = 50
x0 = backward[offset_0::window]
y0 = forward[offset_0::window]
x1 = backward[offset_1::window]
y1 = forward[offset_1::window]
text_0 = [f"Index: {offset_0 + i * window}, bw: {x0_}, fw: {y0[i]}" for i, x0_ in enumerate(x0)]
text_1 = [f"Index: {offset_1 + i * window}, bw: {x1_}, fw: {y1[i]}" for i, x1_ in enumerate(x1)]

min_x = min(min(x0), min(x1))
max_x = max(max(x0), max(x1))

m0 = LinearRegression()
m0.fit(x0.values.reshape([-1, 1]), y0)

m1 = LinearRegression()
m1.fit(x1.values.reshape([-1, 1]), y1)

fig = go.Figure(layout_title="Comparable dispersion despide large R2-difference for offsets 0 and 50",
                layout_xaxis_title="Backward standard deviation",
                layout_yaxis_title="Forward standard deviation")
fig.add_trace(go.Scatter(x=x0, y=y0, mode="markers", name=f"Offset {offset_0}", marker={"color": "#1f77b4"},
                         text=text_0))
fig.add_trace(go.Scatter(x=x1, y=y1, mode="markers", name=f"Offset {offset_1}", marker={"color": "#ff7f0e"},
                        text=text_1))
fig.add_trace(go.Scatter(x=[min_x, max_x], y=[min_x, max_x], 
                         line={"color": "#aaaaaa"}, name="1:1-line", mode="lines"))
fig.add_trace(go.Scatter(x=[min_x, max_x], y=[m0.intercept_ + m0.coef_[0] * min_x, m0.intercept_ + m0.coef_[0] * max_x], 
                         line={"color": "#1f77b4",  "dash":"dash"}, mode="lines", showlegend=False))
fig.add_trace(go.Scatter(x=[min_x, max_x], y=[m1.intercept_ + m1.coef_[0] * min_x, m1.intercept_ + m1.coef_[0] * max_x], 
                         line={"color": "#ff7f0e",  "dash":"dash"}, mode="lines", showlegend=False))
iplot(fig)


It becomes clear that pearson correlation may not be an ideal choice for this kind of analysis. When looking at tho two sets of points, the dispersion seems comparable and I am convinced that the outliers dominate the residual sums of squares. So probably a more robust measure of correlation may improve things.

In [ ]:
def get_sr2(offset: int, window: int):
    return spearmanr(backward[offset::window], forward[offset::window]).correlation

window = 252
fig = go.Figure(layout_yaxis_range=[0,1], 
                layout_title="Spearmans rho less sensitive to window offset than R2",
                layout_xaxis_title="Offset (in trading days)",
                layout_yaxis_title="Spearman's rho")
fig.add_trace(go.Scatter(x=list(range(window)), y=[get_sr2(i, window) for i in range(window)], mode="markers+lines"))
iplot(fig)

The statement that the choice of window offset does not impact further analysis is not correct. If standard OLS is used to choose the best backward window size to predict the volatility in the future one may incur significant distortions depending on the window used.

However, the statement that the choice of window offset has a significant impact on the predictive power seems equally tenuous, since the dispersion (if measured using rank correlations) is fairly stable.

The problems arise with large spikes in volatility that seem to be unpredictable as well as short-lived. Neither ignoring them (R2-Problem) nor deleting those data seems to be a good option. Instead I propose to use a more robust regression.

I will consider RANSAC and Huber regression, choosing the one with less volatility of the parameters over time (and again, if this were a real exercise, the same would have to be done for all combinations of forward and backward windows to ensure that the finding is not an artifact of the one pair chosen for this analysis).

In [ ]:
window = 252
huber = HuberRegressor()
ransac = RANSACRegressor()
ols = LinearRegression()

def get_parameters(offset: int, window: int) -> tuple:
    """Return Huber-intercept, Huber beta, RANSAC-intercept and RANSAC-beta"""
    x = backward[offset::window].values.reshape([-1, 1])
    y = forward[offset::window]
    huber.fit(x, y)
    ransac.fit(x, y)
    ols.fit(x, y)
    return np.array([huber.intercept_, huber.coef_[0], 
                     ransac.estimator_.intercept_, ransac.estimator_.coef_[0],
                     ols.intercept_, ols.coef_[0]]).reshape([1, -1])

coefs = np.concatenate([get_parameters(i, window) for i in range(window)])

fig = go.Figure(layout_yaxis_range=[0,1], 
                layout_title="Huber Regression more stable, but still with significant variability",
                layout_xaxis_title="Offset (in trading days)",
                layout_yaxis_title="Coefficients")

fig.add_trace(go.Scatter(x=np.arange(window), y=coefs[:, 0], line={"color": "#1f77b4", "dash":"dash"}, name="Intercept Huber"))
fig.add_trace(go.Scatter(x=np.arange(window), y=coefs[:, 1], line={"color": "#1f77b4", "dash":"solid"}, name="Coef Huber"))
fig.add_trace(go.Scatter(x=np.arange(window), y=coefs[:, 2], line={"color": "#7f7f7f", "dash":"dash"}, name="Intercept RANSAC"))
fig.add_trace(go.Scatter(x=np.arange(window), y=coefs[:, 3], line={"color": "#7f7f7f", "dash":"solid"}, name="Coef RANSAC"))
fig.add_trace(go.Scatter(x=np.arange(window), y=coefs[:, 4], line={"color": "#2ca02c", "dash":"dash"}, name="Intercept OLS"))
fig.add_trace(go.Scatter(x=np.arange(window), y=coefs[:, 5], line={"color": "#2ca02c", "dash":"solid"}, name="Coef OLS"))
iplot(fig)

Ignoring the (highly volatile) RANSAC, I am somewhat surprised at the fact that the outliers affect the parameters of the regression to a lesser degree than the window offset. Also it is noteworthy that the coefficient is markedly lower than 1 for most windows which is somewhat at odds with the expectations. One explanation could be the spikedness of high-volatility phases: In phases where backwards-volatility is particularly high, the forward volatility is lower than the backward volatility which would explain the size of the coefficient being less than one.

To come to a conclusion about finding "the best" way of predicting the volatility in a given future window: Parameters based on linear estimates between forward and backward volatilities are highly dependent on the window offset and it is not obvious how to choose a point estimator this way. An obvious solution would be to allow for some non-linear dependency between the backwards volatility and the forward volatility. One way would be to apply a log transform to the predictor, another would be to add polynomial terms. Let's try both.

In [ ]:
window = 252
offset_0 = 0
offset_1 = 171
x0 = backward[offset_0::window].values.reshape([-1, 1])
y0 = forward[offset_0::window]
x1 = backward[offset_1::window].values.reshape([-1, 1])
y1 = forward[offset_1::window]

all_x = np.concatenate([x0, x1])
min_x = all_x.min()
max_x = all_x.max()
x_pred = np.linspace(min_x, max_x, 200).reshape([-1, 1])

poly_trafo = PolynomialFeatures(degree=4)

m = LinearRegression()

m.fit(poly_trafo.fit_transform(x0), y0)
m0_pred_poly = m.predict(poly_trafo.fit_transform(x_pred))
m.fit(np.log(x0), y0)
m0_pred_log = m.predict(np.log(x_pred))

m.fit(poly_trafo.fit_transform(x1), y1)
m1_pred_poly = m.predict(poly_trafo.fit_transform(x_pred))
m.fit(np.log(x1), y1)
m1_pred_log = m.predict(np.log(x_pred))



col_0 = "#1f77b4"
col_1 = "#ff7f0e"

fig = go.Figure(layout_title="Comparable dispersion despide large R2-difference for offsets 0 and 50",
               layout_xaxis_title="Backward standard deviation",
               layout_yaxis_title="Forward standard deviation",
               layout_yaxis_range=[0,0.015])
fig.add_trace(go.Scatter(x=x0.flatten(), y=y0, mode="markers", name=f"Offset {offset_0}", marker={"color": col_0},
                        text=text_0))
fig.add_trace(go.Scatter(x=x1.flatten(), y=y1, mode="markers", name=f"Offset {offset_1}", marker={"color": col_1},
                       text=text_1))

fig.add_trace(go.Scatter(x=[min_x, max_x], y=[min_x, max_x],
                        line={"color": "#aaaaaa"}, name="1:1-line", mode="lines"))
fig.add_trace(go.Scatter(x=x_pred.flatten(), 
                         y=m0_pred_poly,
                         line={"color": col_0,  "dash":"dash"}, mode="lines", name="Polynomial"))
fig.add_trace(go.Scatter(x=x_pred.flatten(), 
                        y=m0_pred_log, 
                        line={"color": col_0,  "dash":"dot"}, mode="lines", name="Log trafo"))
fig.add_trace(go.Scatter(x=x_pred.flatten(), 
                         y=m1_pred_poly,
                         line={"color": col_1,  "dash":"dash"}, mode="lines", name="Polynomial"))
fig.add_trace(go.Scatter(x=x_pred.flatten(), 
                        y=m1_pred_log, 
                        line={"color": col_1,  "dash":"dot"}, mode="lines", name="Log trafo"))


iplot(fig)


As expected, polynomial fits behave unpredictably towards outliers and the comparison of how strong coefficients react to window offsets will only be done for the (Huberised) log-transformed model.

In [ ]:
window = 252
huber = HuberRegressor()
huber2 = HuberRegressor()

def get_parameters_log(offset: int, window: int) -> tuple:
    """Return Huber-intercept, Huber beta, RANSAC-intercept and RANSAC-beta"""
    x = backward[offset::window].values.reshape([-1, 1])
    y = forward[offset::window]
    huber.fit(x, y)
    huber2.fit(np.log(x), y)
    return np.array([huber.intercept_, huber.coef_[0], 
                     huber2.intercept_, huber2.coef_[0], ]).reshape([1, -1])

coefs = np.concatenate([get_parameters_log(i, window) for i in range(window)])

fig = go.Figure(layout_title="Parameters of model on transformed data less volatile in absolute terms",
                layout_xaxis_title="Offset (in trading days)",
                layout_yaxis_title="Coefficients")

fig.add_trace(go.Scatter(x=np.arange(window), y=coefs[:, 0], line={"color": col_0, "dash":"dash"}, name="Intercept Untransformed"))
fig.add_trace(go.Scatter(x=np.arange(window), y=coefs[:, 1], line={"color": col_0, "dash":"solid"}, name="Coef Untransformed"))
fig.add_trace(go.Scatter(x=np.arange(window), y=coefs[:, 2], line={"color": col_1, "dash":"dash"}, name="Intercept Transformed"))
fig.add_trace(go.Scatter(x=np.arange(window), y=coefs[:, 3], line={"color": col_1, "dash":"solid"}, name="Coef Transformed"))
iplot(fig)

While in absolute terms the fluctuations of the parameters did not change by much, in relative terms the situation did not get much better. However, maybe this was the wrong way of looking at the problem: While from a modelling point of view (and for the confidence in the model) it would of course be very desirable to have stable model parameters, maybe in practice the stability of the results are more important. As a last analysis before actually finding a good choice of window to predict a given future volatility, I will look at the variability of prediction depending on the window offset.

For that I will outline an area marking the interquartile range as well as lines for the median and the 5% and 95% quantiles for every point for which I predict both untransformed and log-transformed inputs.

In [ ]:
window = 252
huber = HuberRegressor()
huber2 = HuberRegressor()
x_pred = np.linspace(min(min(forward), min(backward)), max(max(forward), max(backward)), 200).reshape([-1, 1])



def get_parameters_log(offset: int, window: int) -> tuple:
    """Return Huber-intercept, Huber beta, RANSAC-intercept and RANSAC-beta"""
    x = backward[offset::window].values.reshape([-1, 1])
    y = forward[offset::window]
    huber.fit(x, y)
    untransformed = huber.predict(x_pred).reshape([-1, 1, 1]) # Dims: x, offset, model
    huber2.fit(np.log(x), y)
    transformed = huber2.predict(np.log(x_pred)).reshape([-1, 1, 1])
    return np.concatenate([untransformed, transformed], axis = 2)

preds = np.concatenate([get_parameters_log(i, window) for i in range(window)], axis=1)
quantiles = np.quantile(preds, [0.05, 0.25, 0.5, 0.75, 0.95], axis=1)



In [ ]:
x_obs = np.linspace(x_pred.min(), x_pred.max(), 30)
bins = np.digitize(backward, x_obs)
observed = (pd.DataFrame({"bin": bins, "fw": forward})
            .groupby("bin")["fw"]
            .quantile(q=[0.05, 0.25, 0.5, 0.75, 0.95])
            .unstack(level=1))

In [ ]:
col_0 = "rgba(31,119,180, 0.2)"
col_1 = "rgba(255,127,14, 0.2)"
gray = "rgba(70, 70, 70, 0.2)"

fig = go.Figure(layout_title="Overall fit is hard to judge",
                layout_xaxis_title="Past volatility",
                layout_yaxis_title="Predicted future volatility")

fig.add_trace(go.Scatter(x=x_obs, y=observed[0.05].values, line={"color": gray, "dash":"solid"}, showlegend=False))
fig.add_trace(go.Scatter(x=x_obs, y=observed[0.50].values, line={"color": gray, "dash":"solid"}, showlegend=False))
fig.add_trace(go.Scatter(x=x_obs, y=observed[0.95].values, line={"color": gray, "dash":"solid"}, showlegend=False))
fig.add_trace(go.Scatter(x=x_obs, y=observed[0.25].values, line={"color": gray, "dash":"solid"}, showlegend=False))
fig.add_trace(go.Scatter(x=x_obs, y=observed[0.75].values, line={"color": gray, "dash":"solid"}, name="Observed", fill="tonexty"))

fig.add_trace(go.Scatter(x=x_pred.flatten(), y=quantiles[0, :, 0], line={"color": col_0, "dash":"solid"}, showlegend=False))
fig.add_trace(go.Scatter(x=x_pred.flatten(), y=quantiles[2, :, 0], line={"color": col_0, "dash":"solid"}, showlegend=False))
fig.add_trace(go.Scatter(x=x_pred.flatten(), y=quantiles[4, :, 0], line={"color": col_0, "dash":"solid"}, showlegend=False))
fig.add_trace(go.Scatter(x=x_pred.flatten(), y=quantiles[1, :, 0], line={"color": col_0, "dash":"solid"}, showlegend=False))
fig.add_trace(go.Scatter(x=x_pred.flatten(), y=quantiles[3, :, 0], line={"color": col_0, "dash":"solid"}, name="Pred Untransformed", fill="tonexty"))


fig.add_trace(go.Scatter(x=x_pred.flatten(), y=quantiles[0, :, 1], line={"color": col_1, "dash":"solid"}, showlegend=False))
fig.add_trace(go.Scatter(x=x_pred.flatten(), y=quantiles[2, :, 1], line={"color": col_1, "dash":"solid"}, showlegend=False))
fig.add_trace(go.Scatter(x=x_pred.flatten(), y=quantiles[4, :, 1], line={"color": col_1, "dash":"solid"}, showlegend=False))
fig.add_trace(go.Scatter(x=x_pred.flatten(), y=quantiles[1, :, 1], line={"color": col_1, "dash":"solid"}, showlegend=False))
fig.add_trace(go.Scatter(x=x_pred.flatten(), y=quantiles[3, :, 1], line={"color": col_1, "dash":"solid"}, name="Pred Transformed", fill="tonexty"))
iplot(fig)

The number of observations is limited and the true volatility of observed values, in particular for higher past volatilities is likely to be understated. While predictions on transformed data likely underpredict future volatilities if the past was marked by really low volatilities, predictions on transformed data seem to give more credible results for higher past volatility regimes. While in practice getting this exactly right (and experimenting much more with proper predictions based on more than just one input variable would be required), I will leave it at that for now and trust the book for now.

### Volatility growth

In all of the following i disregards the days of the week, holidays etc. and treat the data as a steady stream of trading days. While not entirely accurate this seems somewhat justified from the analysis above and common practice (cf. Hull).

Let $N$ be the number of observed trading days, $\{x_0, ..., x_{N-1}\}$ be the observed log returns, $w \in \mathbb{N}_+$ the window size, and $t \in \{w, 1, ..., N-w-1\}$ be a time point at which the volatility is observed. Let $\hat{\sigma}_{t}^{w} := \sqrt{\frac{1}{w} \cdot \sum_{i=t-w+1}^{t}(x_i - \bar{x}_t)^2}$ with $\bar{x}_t := \frac{1}{w} \cdot \sum_{i=t-w+1}^{t}x_i$.

Assuming the daily log returns follow a zero centred normal distribution with standard deviation $\hat{\sigma}_{t}^{w}$, I can normalise these forward returns to make them all standard normal and hence comparable. The expectation is then that $Y_{t, j}:=\sum_{k=1}^{j}x_{n_t + k} \sim \mathcal{N(0, j)}$. To verify this, I will have to choose the $w$ and the $t$ such that the sample size is large enough (small $w$) but the $t$ are far enough apart such that the dependence is not too bad.

Before having done the analysis my expectation is that the lower tail of the distribution is heavier than the upper tail (big moves tend to be to the downside), and that it is leptokurtic (movements are flat followed by larger movements rather than a steady creep upwards). 

I will test different sizes for $w$, but have the windows overlap, such that the evaluation period of one $t$ is the data on which the standard deviation of the next window is calculated.

In [ ]:
def get_observed_returns(w, correct: bool = True) -> np.ndarray:
    """Gets all valid cumulative log returns"""
    returns = spx_hist["log_return"]
    if correct:
        returns = returns - returns.mean()
    backward = returns.rolling(w).std().values
    forward = np.concatenate([
        ((returns
          .rolling(pd.api.indexers.FixedForwardWindowIndexer(window_size=n))
          .sum()) 
         - returns)
        .values
        .reshape([-1, 1])
        for n in range(2, w+2)],
        axis=1)
    forward_norm = forward / np.tile(backward.reshape([-1, 1]), (1, w))
    forward_norm = forward_norm[~np.isnan(forward_norm).any(axis=1)]
    return forward_norm

def select_observed_returns(forward_norm: np.ndarray, w: int, offset=0) -> np.ndarray:
    """Selects log returns so that they become less correlated"""
    return forward_norm[offset::w, :]

def get_quantiles(forward_norm: np.ndarray, quantiles: list) -> np.ndarray:
    """Calculates quantiles from the observed returns (to be compared with the normal quantiles)"""
    return np.quantile(forward_norm, quantiles, axis=0)

def get_window_quantiles(w: int, offset=0, correct: bool = True, quantiles=[0.01, 0.05, 0.25, 0.5, 0.75, 0.95, 0.99]):
    cum_returns = get_observed_returns(w, correct)
    cum_norm_returns = select_observed_returns(cum_returns, w, offset=offset)
    return get_quantiles(cum_norm_returns, quantiles), cum_norm_returns.shape[0]

def get_normal_quantiles(t_max: int, quantiles: list = [0.01, 0.05, 0.25, 0.5, 0.75, 0.95, 0.99]) -> np.ndarray:
    """Returns theoretical quantiles from the standard normal"""
    q = norm.ppf(quantiles).reshape([-1, 1])
    scale = np.array([np.sqrt(i + 1) for i in range(t_max)]).reshape([1, -1])
    return np.matmul(q, scale)

def add_traces(fig, quantiles: np.ndarray, col: str, fillcol: str, name: str):
    """Adds quantile traces to the fig and returns the fig. Assumes there are 7 quantiles to show with 2-4 in colors"""
    fig.add_trace(go.Scatter(x=[i + 1 for i in range(quantiles.shape[1])], y=quantiles[0, :], line={"color": col, "dash":"dot"}, showlegend=False))
    fig.add_trace(go.Scatter(x=[i + 1 for i in range(quantiles.shape[1])], y=quantiles[6, :], line={"color": col, "dash":"dot"}, showlegend=False))
    fig.add_trace(go.Scatter(x=[i + 1 for i in range(quantiles.shape[1])], y=quantiles[1, :], line={"color": col, "dash":"dash"}, showlegend=False))
    fig.add_trace(go.Scatter(x=[i + 1 for i in range(quantiles.shape[1])], y=quantiles[5, :], line={"color": col, "dash":"dash"}, showlegend=False))
    fig.add_trace(go.Scatter(x=[i + 1 for i in range(quantiles.shape[1])], y=quantiles[3, :], line={"color": col, "dash":"solid"}, showlegend=False))
    fig.add_trace(go.Scatter(x=[i + 1 for i in range(quantiles.shape[1])], y=quantiles[2, :], line={"color": "rgba(0, 0, 0, 0)", "dash":"solid"}, showlegend=False))
    fig.add_trace(go.Scatter(x=[i + 1 for i in range(quantiles.shape[1])], y=quantiles[4, :], line={"color": "rgba(0, 0, 0, 0)", "dash":"solid"}, name=name, fill="tonexty", fillcolor=fillcol))
    return fig


In [ ]:
w = 21*6


uncorrected_window_quantiles, _ = get_window_quantiles(w, correct=False)
corrected_window_quantiles, n = get_window_quantiles(w)

col_0 = "rgba(31,119,180, 0.6)"
col_0_f = "rgba(31,119,180, 0.3)"
col_1 = "rgba(255,127,14, 0.8)"
col_1_f = "rgba(255,127,14, 0.4)"
gray = "rgba(90, 90, 90, 0.8)"
gray_f = "rgba(90, 90, 90, 0.4)"


fig = go.Figure(layout_title=f"True development too positive, smaller IQR and unexpected tails, w={w}, n={n}",
                layout_xaxis_title="Trading days after t",
                layout_yaxis_title="Cumulative normalised return")

fig = add_traces(fig, get_normal_quantiles(w), gray, gray_f, "Normal")
fig = add_traces(fig, uncorrected_window_quantiles, col_0, col_0_f, "Observed Uncorrected")
fig = add_traces(fig, corrected_window_quantiles, col_1, col_1_f, "Observed Corrected")

iplot(fig)

The expectations were partly met. First, it has to be noted that there is a central trend in the returns (after all, we expect stocks to have positive returns over the long run) which explains why there uncorrected returns deviate from the zero centred normal assumption by rising over time. I therefore added a correction term and subtracted mean and median (here only the mean is shown). After this correction the central tendency is a surprisingly good fit for the normal when it comes to the quartiles.

That said, the tails do not seem to follow the normal assumption. As expected the lower tails are heavier. However, while the uncorrected observations have an upper tails that looks about right, the distance between the median and that upper tail is too narrow, as evidenced by the too light tails of the corrected graph. Still, with such a limited sample size it is hard to make statements about the tails.

I tested several different window sizes and for all of them the above holds. Also I tested accepting the correlated measures of performing the calculations on every calendar day instead of in intervals of $w$ and got qualitatively similar resuts. I was surprised at their robustnes.

Of course it has to be said that SPX is an extremely liquid index and that individual firms can be expected to show very different behaviour.

### Implied volatilities